In this notebook I will be exploring the NBADATA.csv file, and trying to identify the relevant statistics for modelling. This will be split into two sections. 


1. Which stats are the most correlated to victory? 
    By looking at all of these box scores, we can see what the game's final point differential (+/-) was. Using this as an output along with all of the box score statistics, we can infer which values are the most correlated to it, along with other useful takeaways to be described concurrently with the study. 
    

2. How many games are sufficient for lookback? 
    Obviously we cannot use the ingame data for prediction purposes, so some sort of lookback prediction proxy must be used. In order to capture what the teams expected performance will be in the game for prediction, a lookback window will be used over the previous x games to serve as that team's forecasted stats. I will try to identify how many games of lookback is the best for this purpose. 

In [1]:
#import dependencies, and dataset. 


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline



In [2]:
filename = 'NBADATA.csv'

nba_dataset = pd.read_csv('NBADATA.csv')

In [3]:
nba_dataset.head()

,Unnamed: 0,GAME_ID,Date,Team,Home,Away,FG,FGA,3P,3PA,...,AST,PF,ST,TO,BL,PTS,SPREAD,PLUS_MINUS,OU,TOTAL
0,4,21300001,10/29/13,Orlando Magic,0,1,36,93,9,19,...,17,26,10,17,6,87,12.0,-10,U,189.5
1,5,21300001,10/29/13,Indiana Pacers,1,0,34,71,7,17,...,17,13,4,20,18,97,-12.0,10,U,189.5
2,1,21300002,10/29/13,Chicago Bulls,0,1,35,83,7,26,...,23,27,11,18,4,95,5.0,-12,O,188.0
3,0,21300002,10/29/13,Miami Heat,1,0,37,72,11,20,...,26,21,10,18,7,107,-5.0,12,O,188.0
4,3,21300003,10/29/13,Los Angeles Clippers,0,1,41,83,8,21,...,27,21,11,16,4,103,-9.5,-13,O,201.5


# Exploratory Analysis

Just want the basic box score stuff here, see if generic box score things contribute to the PLUS_MINUS category, which corresponds to the final point differential of the game. As a sanity check, the spread should play one of the biggest roles, as vegas bookmakers are smart and typically keep it close. 

In [4]:
#delete irrelevant columns (for this section), they will be useful for other pieces of the study but not here!
nba_explore = nba_dataset
del nba_explore['Unnamed: 0'],nba_explore['GAME_ID'],nba_explore['Date'],nba_explore['Team'],nba_explore['Home'],nba_explore['Away']
del nba_explore['OU'],nba_explore['TOTAL']

#add some other potential columns, like efficency. 
nba_explore['3P%'] = np.divide(nba_explore['3P'].values,nba_explore['3PA'].values) 

nba_explore['FG%'] = np.divide(nba_explore['FG'].values,nba_explore['FGA'].values)
nba_explore['FT%'] = np.divide(nba_explore['FT'].values,nba_explore['FTA'].values)
nba_explore['TRB']  = nba_explore['OR'] + nba_explore['DR']

nba_explore['AST/TO'] = np.divide(nba_explore['AST'].values,nba_explore['TO'].values)


relevant_stats = []
for col in nba_explore.columns:
    if col != 'PLUS_MINUS':
        print(col + " Correlation to Outcome")
        corr = np.corrcoef(nba_explore[col],nba_explore['PLUS_MINUS'])
        print(corr[0][1])
        if abs(corr[0][1]) < .1:
            pass
        else:
            relevant_stats.append(col)
        


FG Correlation to Outcome
0.503997991165
FGA Correlation to Outcome
-0.000850904870708
3P Correlation to Outcome
0.330232588337
3PA Correlation to Outcome
0.0879946317693
FT Correlation to Outcome
0.110525621062
FTA Correlation to Outcome
0.0682632854309
OR Correlation to Outcome
-0.0280868972788
DR Correlation to Outcome
0.419821151912
AST Correlation to Outcome
0.409512742091
PF Correlation to Outcome
-0.0607043277291
ST Correlation to Outcome
0.165059256663
TO Correlation to Outcome
-0.131423640983
BL Correlation to Outcome
0.196087441626
PTS Correlation to Outcome
0.567488838081
SPREAD Correlation to Outcome
-0.493897887864
3P% Correlation to Outcome
0.389249598123
FG% Correlation to Outcome
0.548656258878
FT% Correlation to Outcome
0.127611348862
TRB Correlation to Outcome
0.328001312084
AST/TO Correlation to Outcome
0.304760416223


In [5]:
relevant_stats

['FG',
 '3P',
 'FT',
 'DR',
 'AST',
 'ST',
 'TO',
 'BL',
 'PTS',
 'SPREAD',
 '3P%',
 'FG%',
 'FT%',
 'TRB',
 'AST/TO']

So now I've identified which stats I want to proceed with. The next step is to determine the optimal lookback window size. This will be based on a summed correlation coeffecient, by trying to study the correlation of statistic x with that team's previous y game average with statistic x, and determine which window size fits best. 

Out of convenience, I'm still working with one team at a time, and summing up over all. To be explained better below!

In [7]:
#reread in the data, really not sure why deleting the cols from one dataset deletes it from all others. 
nba_dataset = pd.read_csv('NBADATA.csv')

In [8]:
def rolling_correlations(data,ngames):
    """Determines the correlation between a team's performance in a
    particular category compared to that team's split average over the previous ngames. 
    Use this function to determine optimal lookback period. 
    
    data : df
        Dataframe containing all the box scores over previous 5 season. 
        
    ngames : int
        Lookback splits. Ex: ngames=5 corresponds to team's splits over previous 5 games. 
    """
#delete irrelevant columns (for this section), they will be useful for other pieces of the study but not here!
    nba_explore = pd.read_csv('NBADATA.csv')
    del nba_explore['Unnamed: 0'],nba_explore['GAME_ID'],nba_explore['Date'],nba_explore['Team'],nba_explore['Home'],nba_explore['Away']
    del nba_explore['OU'],nba_explore['TOTAL']

#add some other potential columns, like efficency. 
    nba_explore['3P%'] = np.divide(nba_explore['3P'].values,nba_explore['3PA'].values) 

    nba_explore['FG%'] = np.divide(nba_explore['FG'].values,nba_explore['FGA'].values)
    nba_explore['FT%'] = np.divide(nba_explore['FT'].values,nba_explore['FTA'].values)
    nba_explore['TRB']  = nba_explore['OR'] + nba_explore['DR']

    nba_explore['AST/TO'] = np.divide(nba_explore['AST'].values,nba_explore['TO'].values)


    relevant_stats = []
    for col in nba_explore.columns:
        if col != 'PLUS_MINUS':
           # print(col + " Correlation to Outcome")
            corr = np.corrcoef(nba_explore[col],nba_explore['PLUS_MINUS'])
            #print(corr[0][1])
            if abs(corr[0][1]) < .1:
                pass
            else:
                relevant_stats.append(col)
        

    print("Using a window of " ,ngames, " games. ")
    data['AST/TO'] = np.divide(data['AST'].values,data['TO'].values)
    data['3P%'] = np.divide(data['3P'].values,data['3PA'].values) 
    data['FG%'] = np.divide(data['FG'].values,data['FGA'].values)
    data['FT%'] = np.divide(data['FT'].values,data['FTA'].values)
    del data['Unnamed: 0'],data['TOTAL']
   # del data['Date']
    data = data.loc[data['GAME_ID'].values < 41300001] #genius! No playoff games now :)   
    #del data['Team'] 
    #data = pd.get_dummies(data) #sometimes option to hot tcode team, but not yet. Seems like overfitting. 
    teams = data.Team.unique() #each nba team. 
#iterate over those teams, make a rolling window
    nba_data = pd.DataFrame([])
    season_ids = []
    for i,val in enumerate(data['GAME_ID'].values):  #loop through every game
        season_ids.append(str(val)[1:3])

    data['Season_ID'] = season_ids #identify the unique seasons. 

    for team in teams:  #for each team
       # print(team)
    #get separate seasons here
        team_data = data.loc[data['Team'] == team]  #this contains the box score of every team game from 2013 to 2018.
        for season in data['Season_ID'].unique(): #this contains the box score of that team for that season. 
            #print(season)
            team_season = team_data.loc[team_data['Season_ID'] == season]
        
            stuff_to_turn_into_avgs =  relevant_stats  #['OR', 'DR', 'TOT', 'PF', 'ST', 'TO', 'BL', '3P%', 'FG%', 'FT%']
            for col in team_season.columns:
                if col in stuff_to_turn_into_avgs:
                        team_season['Rolling ' + col] = team_season[col].rolling(window=ngames).mean().shift(1)

            #split each season up here, 
                    #if col != 'PTS':
                    #    team_season['Rolling ' + col] = team_season[col].rolling(window=N_GAMES).mean().shift(1)

                        del team_season[col]
                    
            nba_data =  nba_data.append(team_season)

           # df = pd.concat([road_df,home_df],axis=1)
#reorganize the dataset. 
    nba_data_splits = nba_data.sort_values(by = ['GAME_ID', 'Home','Away'], ascending=[True, True,False])

    nba_data_splits.dropna(inplace=True)

    del nba_data_splits['Home'], nba_data_splits['Away'], nba_data_splits['FGA'], nba_data_splits['3PA'], nba_data_splits['FTA'], nba_data_splits['OR'],nba_data_splits['PF']                                                                                                                                
    del nba_data_splits['PLUS_MINUS'], nba_data_splits['OU'],nba_data_splits['Rolling SPREAD'],nba_data_splits['Season_ID']
    nba_dataset = pd.read_csv('NBADATA.csv')                                                                                                               
    rolling_vals = nba_data_splits
  #  print(rolling_vals.head())
    game_vals = nba_dataset
    merged = game_vals.merge(rolling_vals,on=['GAME_ID','Team'])
    print(rolling_vals.columns)
#   del merged['GAME_ID'],merged['Team'],merged['Home'],merged['Away'],merged['SPREAD']
  #  print("do dates align? ")
 #   print(merged[['Date_x','Date_y']].values[0:10])
    total_corr = 0
    for col in merged:
        try:
            corr = np.corrcoef(merged[col],merged['Rolling ' + col])[0][1]
            #print(col)
            #print(corr)
            total_corr += abs(corr)
        except:
            pass
    return total_corr

In [ ]:
ngames = [1,5,10,15,30,50,75,81]
ngame_corrs = []
for lookback in ngames:
    nba_dataset = pd.read_csv('NBADATA.csv')
    total_corr = rolling_correlations(nba_dataset,lookback)
    print("Total Correlation = ", total_corr)
    ngame_corrs.append(total_corr)
  

In [ ]:
  
plt.title('Correlation Between Splits and in Game Box Score')   
plt.xlabel('N Games')
plt.ylabel('Correlation')
plt.style.use('ggplot')
plt.plot(ngames,ngame_corrs,'bo--')

# Going to stop here for the day. The next step is combing the rolling columns here with the relevant stats of the previous one, make sure the game_ids are aligned, and confirm some sense of correlation between the two. Generalize it enough so it can be done for an arbitrary window size to compare these, and also maybe see if these columns still posess a correlaiton to the outcome as well? After all that is the point of this study. 

In [9]:
def create_dataset(data,ngames):
    """Using all the other information acquired above, create the datset using the relevant categories stated. 
    
     data : dataframe
         The NBADATA dataframe. 
    """
    nba_explore = pd.read_csv('NBADATA.csv')
    del nba_explore['Unnamed: 0'],nba_explore['GAME_ID'],nba_explore['Date'],nba_explore['Team'],nba_explore['Home'],nba_explore['Away']
    del nba_explore['OU'],nba_explore['TOTAL']

#add some other potential columns, like efficency. 
    nba_explore['3P%'] = np.divide(nba_explore['3P'].values,nba_explore['3PA'].values) 

    nba_explore['FG%'] = np.divide(nba_explore['FG'].values,nba_explore['FGA'].values)
    nba_explore['FT%'] = np.divide(nba_explore['FT'].values,nba_explore['FTA'].values)
    nba_explore['TRB']  = nba_explore['OR'] + nba_explore['DR']

    nba_explore['AST/TO'] = np.divide(nba_explore['AST'].values,nba_explore['TO'].values)


    relevant_stats = []
    for col in nba_explore.columns:
        if col != 'PLUS_MINUS':
           # print(col + " Correlation to Outcome")
            corr = np.corrcoef(nba_explore[col],nba_explore['PLUS_MINUS'])
            #print(corr[0][1])
            if abs(corr[0][1]) < .1:
                pass
            else:
                relevant_stats.append(col)
        

    data['AST/TO'] = np.divide(data['AST'].values,data['TO'].values)
    data['3P%'] = np.divide(data['3P'].values,data['3PA'].values) 
    data['FG%'] = np.divide(data['FG'].values,data['FGA'].values)
    data['FT%'] = np.divide(data['FT'].values,data['FTA'].values)
    del data['Unnamed: 0'],data['TOTAL']
   # del data['Date']
    data = data.loc[data['GAME_ID'].values < 41300001] #genius! No playoff games now :)   
    #del data['Team'] 
    #data = pd.get_dummies(data) #sometimes option to hot tcode team, but not yet. Seems like overfitting. 
    teams = data.Team.unique() #each nba team. 
#iterate over those teams, make a rolling window
    nba_data = pd.DataFrame([])
    season_ids = []
    for i,val in enumerate(data['GAME_ID'].values):  #loop through every game
        season_ids.append(str(val)[1:3])

    data['Season_ID'] = season_ids #identify the unique seasons. 

    for team in teams:  #for each team
       # print(team)
    #get separate seasons here
        team_data = data.loc[data['Team'] == team]  #this contains the box score of every team game from 2013 to 2018.
        for season in data['Season_ID'].unique(): #this contains the box score of that team for that season. 
            #print(season)
            team_season = team_data.loc[team_data['Season_ID'] == season]
        
            stuff_to_turn_into_avgs =  relevant_stats  #['OR', 'DR', 'TOT', 'PF', 'ST', 'TO', 'BL', '3P%', 'FG%', 'FT%']
            for col in team_season.columns:
                if col in stuff_to_turn_into_avgs:
                        team_season['Rolling ' + col] = team_season[col].rolling(window=ngames).mean().shift(1)

            #split each season up here, 
                    #if col != 'PTS':
                    #    team_season['Rolling ' + col] = team_season[col].rolling(window=N_GAMES).mean().shift(1)

                        del team_season[col]
                    
            nba_data =  nba_data.append(team_season)

           # df = pd.concat([road_df,home_df],axis=1)
#reorganize the dataset. 
    nba_data_splits = nba_data.sort_values(by = ['GAME_ID', 'Home','Away'], ascending=[True, True,False])

    nba_data_splits.dropna(inplace=True)

    del nba_data_splits['FGA'], nba_data_splits['3PA'], nba_data_splits['FTA'], nba_data_splits['OR'],nba_data_splits['PF']                                                                                                                                
    del nba_data_splits['PLUS_MINUS'], nba_data_splits['OU'],nba_data_splits['Rolling SPREAD'],nba_data_splits['Season_ID']
    nba_dataset = pd.read_csv('NBADATA.csv')                                                                                                               
    rolling_vals = nba_data_splits
    
    
    return rolling_vals

In [10]:
data = pd.read_csv('NBADATA.csv')
rolling_vals = create_dataset(data,30)

/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
spreads = pd.read_csv('NBADATA.csv')
spreads = spreads[['GAME_ID','SPREAD','Team']]

In [12]:
#spreads

In [13]:
test = rolling_vals.merge(spreads,on=['GAME_ID','Team'])

In [14]:
test.sort_values(by=['GAME_ID'])

,GAME_ID,Date,Team,Home,Away,Rolling FG,Rolling 3P,Rolling FT,Rolling DR,Rolling AST,Rolling ST,Rolling TO,Rolling BL,Rolling PTS,Rolling AST/TO,Rolling 3P%,Rolling FG%,Rolling FT%,SPREAD
0,21300275,4/8/14,San Antonio Spurs,0,1,41.033333,8.966667,17.366667,35.066667,25.966667,7.366667,14.200000,5.766667,108.400000,1.940596,0.401666,0.486488,0.814569,-7.0
1,21300275,4/8/14,Minnesota Timberwolves,1,0,39.166667,7.066667,22.633333,31.366667,24.500000,8.600000,13.600000,3.766667,108.033333,1.916725,0.333378,0.458320,0.766028,7.0
2,21300413,12/23/13,Utah Jazz,0,1,35.366667,6.066667,15.866667,29.166667,19.666667,7.166667,14.700000,4.900000,92.666667,1.518058,0.330579,0.430660,0.754741,6.5
3,21300424,12/26/13,Houston Rockets,1,0,37.833333,9.466667,21.333333,34.633333,20.900000,7.766667,16.233333,6.166667,106.466667,1.387903,0.354389,0.480004,0.699482,-7.5
4,21300426,12/26/13,Los Angeles Clippers,0,1,37.933333,8.000000,21.133333,32.600000,23.666667,8.300000,13.866667,4.600000,105.000000,1.805647,0.332414,0.466906,0.721228,4.5
5,21300428,12/27/13,Detroit Pistons,0,1,38.900000,6.400000,16.600000,30.066667,20.200000,9.000000,14.566667,5.300000,100.800000,1.495818,0.305121,0.453750,0.665598,-4.0
6,21300433,12/27/13,Utah Jazz,1,0,35.333333,6.233333,15.633333,29.033333,19.633333,7.100000,14.400000,4.966667,92.533333,1.541718,0.336766,0.431026,0.761408,-3.5
7,21300435,12/27/13,Golden State Warriors,1,0,38.500000,9.533333,16.033333,34.766667,22.500000,7.733333,16.733333,4.966667,102.566667,1.474916,0.404747,0.460122,0.728847,-6.5
8,21300439,12/28/13,Detroit Pistons,0,1,38.900000,6.266667,16.033333,29.966667,19.933333,9.000000,14.466667,5.333333,100.100000,1.486929,0.301880,0.450768,0.656002,3.5
9,21300440,12/28/13,Charlotte Hornets,0,1,34.466667,4.833333,18.400000,32.633333,19.366667,6.400000,12.600000,5.433333,92.166667,1.722615,0.317372,0.422401,0.718197,6.0


In [15]:
test = test.sort_values(by = ['GAME_ID', 'Home','Away'], ascending=[True, True,False])


In [16]:
from collections import Counter

counts = Counter(test['GAME_ID'].values)

test['GAME_ID'].values

vals = np.array(list(counts.values())) == 2

useable_games = np.array(list(counts.keys()))[vals]

In [17]:
useable_games = np.array(list(counts.keys()))[vals]

In [18]:
test

,GAME_ID,Date,Team,Home,Away,Rolling FG,Rolling 3P,Rolling FT,Rolling DR,Rolling AST,Rolling ST,Rolling TO,Rolling BL,Rolling PTS,Rolling AST/TO,Rolling 3P%,Rolling FG%,Rolling FT%,SPREAD
0,21300275,4/8/14,San Antonio Spurs,0,1,41.033333,8.966667,17.366667,35.066667,25.966667,7.366667,14.200000,5.766667,108.400000,1.940596,0.401666,0.486488,0.814569,-7.0
1,21300275,4/8/14,Minnesota Timberwolves,1,0,39.166667,7.066667,22.633333,31.366667,24.500000,8.600000,13.600000,3.766667,108.033333,1.916725,0.333378,0.458320,0.766028,7.0
2,21300413,12/23/13,Utah Jazz,0,1,35.366667,6.066667,15.866667,29.166667,19.666667,7.166667,14.700000,4.900000,92.666667,1.518058,0.330579,0.430660,0.754741,6.5
3,21300424,12/26/13,Houston Rockets,1,0,37.833333,9.466667,21.333333,34.633333,20.900000,7.766667,16.233333,6.166667,106.466667,1.387903,0.354389,0.480004,0.699482,-7.5
4,21300426,12/26/13,Los Angeles Clippers,0,1,37.933333,8.000000,21.133333,32.600000,23.666667,8.300000,13.866667,4.600000,105.000000,1.805647,0.332414,0.466906,0.721228,4.5
5,21300428,12/27/13,Detroit Pistons,0,1,38.900000,6.400000,16.600000,30.066667,20.200000,9.000000,14.566667,5.300000,100.800000,1.495818,0.305121,0.453750,0.665598,-4.0
6,21300433,12/27/13,Utah Jazz,1,0,35.333333,6.233333,15.633333,29.033333,19.633333,7.100000,14.400000,4.966667,92.533333,1.541718,0.336766,0.431026,0.761408,-3.5
7,21300435,12/27/13,Golden State Warriors,1,0,38.500000,9.533333,16.033333,34.766667,22.500000,7.733333,16.733333,4.966667,102.566667,1.474916,0.404747,0.460122,0.728847,-6.5
8,21300439,12/28/13,Detroit Pistons,0,1,38.900000,6.266667,16.033333,29.966667,19.933333,9.000000,14.466667,5.333333,100.100000,1.486929,0.301880,0.450768,0.656002,3.5
9,21300440,12/28/13,Charlotte Hornets,0,1,34.466667,4.833333,18.400000,32.633333,19.366667,6.400000,12.600000,5.433333,92.166667,1.722615,0.317372,0.422401,0.718197,6.0


In [19]:
#test.loc[test['GAME_ID'].values in useable_games]

In [20]:
len(useable_games)

3850

In [21]:
test.columns

Index(['GAME_ID', 'Date', 'Team', 'Home', 'Away', 'Rolling FG', 'Rolling 3P',
       'Rolling FT', 'Rolling DR', 'Rolling AST', 'Rolling ST', 'Rolling TO',
       'Rolling BL', 'Rolling PTS', 'Rolling AST/TO', 'Rolling 3P%',
       'Rolling FG%', 'Rolling FT%', 'SPREAD'],
      dtype='object')

In [23]:
clunky = pd.DataFrame([])
for col in test.columns:
    clunky[col] = test[col]  #how to assign the same columns 

for i, row in enumerate(clunky.values):
    if row[0] not in useable_games:
        print(i)
        print('invalid')
        clunky = clunky.drop(index=i)

2
invalid
3
invalid
4
invalid
5
invalid
6
invalid
7
invalid
8
invalid
9
invalid
10
invalid
13
invalid
14
invalid
15
invalid
16
invalid
17
invalid
18
invalid
21
invalid
22
invalid
25
invalid
28
invalid
31
invalid
38
invalid
39
invalid
40
invalid
63
invalid
1560
invalid
1561
invalid
1562
invalid
1563
invalid
1564
invalid
1565
invalid
1566
invalid
1567
invalid
1568
invalid
1571
invalid
1572
invalid
1573
invalid
1574
invalid
1575
invalid
1588
invalid
1607
invalid
1610
invalid
1631
invalid
3120
invalid
3121
invalid
3122
invalid
3123
invalid
3124
invalid
3131
invalid
3132
invalid
3133
invalid
3134
invalid
3137
invalid
3138
invalid
3145
invalid
3146
invalid
3147
invalid
3150
invalid
3151
invalid
3152
invalid
3157
invalid
3158
invalid
3159
invalid
3168
invalid
3175
invalid
3176
invalid
3177
invalid
4682
invalid
4685
invalid
4686
invalid
4687
invalid
4688
invalid
4689
invalid
4698
invalid
4699
invalid
4700
invalid
4701
invalid
4702
invalid
4703
invalid
4704
invalid
4709
invalid
4710
invalid
471

In [24]:
clunky

,GAME_ID,Date,Team,Home,Away,Rolling FG,Rolling 3P,Rolling FT,Rolling DR,Rolling AST,Rolling ST,Rolling TO,Rolling BL,Rolling PTS,Rolling AST/TO,Rolling 3P%,Rolling FG%,Rolling FT%,SPREAD
0,21300275,4/8/14,San Antonio Spurs,0,1,41.033333,8.966667,17.366667,35.066667,25.966667,7.366667,14.200000,5.766667,108.400000,1.940596,0.401666,0.486488,0.814569,-7.0
1,21300275,4/8/14,Minnesota Timberwolves,1,0,39.166667,7.066667,22.633333,31.366667,24.500000,8.600000,13.600000,3.766667,108.033333,1.916725,0.333378,0.458320,0.766028,7.0
11,21300447,12/28/13,Utah Jazz,0,1,35.633333,6.366667,15.600000,29.066667,19.800000,7.300000,14.233333,5.033333,93.233333,1.563940,0.340005,0.434021,0.767087,14.5
12,21300447,12/28/13,Los Angeles Clippers,1,0,38.200000,8.000000,20.900000,32.800000,23.566667,8.300000,13.500000,4.633333,105.300000,1.909397,0.333049,0.465288,0.719054,-14.5
19,21300455,12/30/13,Dallas Mavericks,0,1,39.466667,8.533333,16.466667,30.700000,23.233333,9.533333,14.133333,4.700000,103.933333,1.840691,0.381000,0.469546,0.799961,4.0
20,21300455,12/30/13,Minnesota Timberwolves,1,0,38.400000,7.933333,21.700000,32.133333,23.433333,9.300000,13.533333,2.866667,106.433333,1.954478,0.337522,0.429603,0.794908,-4.0
23,21300458,12/30/13,Charlotte Hornets,0,1,34.866667,5.033333,18.500000,33.233333,19.766667,6.333333,12.700000,5.366667,93.266667,1.742615,0.316859,0.424602,0.720772,2.5
24,21300458,12/30/13,Utah Jazz,1,0,35.566667,6.466667,15.533333,29.500000,20.200000,7.300000,14.366667,4.966667,93.133333,1.579379,0.346225,0.434064,0.771320,-2.5
26,21300460,12/31/13,Atlanta Hawks,0,1,39.000000,9.566667,15.866667,32.700000,25.800000,8.300000,14.600000,4.466667,103.433333,1.921392,0.378132,0.468310,0.762918,3.0
27,21300460,12/31/13,Boston Celtics,1,0,36.566667,6.600000,15.500000,31.300000,18.666667,6.800000,15.000000,4.566667,95.233333,1.347540,0.345794,0.454824,0.770221,-3.0


In [25]:
nba_data  = clunky
nba_data_splits = nba_data.sort_values(by = ['GAME_ID', 'Home','Away'], ascending=[True, True,False])


In [26]:
#nba_data_splits

In [27]:
#del nba_data_splits['GAME_ID'],nba_data_splits['Date']
#del nba_data_splits['Home'],nba_data_splits['Away'],nba_data_splits['Team']
 
    #
#Convert to the common box score already used. 

road_df = nba_data_splits.iloc[::2]
home_df = nba_data_splits.iloc[1::2]
for col in nba_data_splits.columns:
    road_df['road_' + col] = road_df[col]
    home_df['home_' + col] = home_df[col]
    
    del road_df[col],home_df[col]

home_df.reset_index(inplace=True)
road_df.reset_index(inplace=True)

#merged into a dataframe here. 
df = pd.concat([road_df,home_df],axis=1)
del df['index']

#create the dataset here. Can consider the spread, or winner. 
#at the moment only using a single classifier, that seems sufficient. A home team loss is synonymous with a road team win. 

#del df['road_PTS'], df['home_PTS'],df['home_SPREAD']

/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [28]:
df

,road_GAME_ID,road_Date,road_Team,road_Home,road_Away,road_Rolling FG,road_Rolling 3P,road_Rolling FT,road_Rolling DR,road_Rolling AST,...,home_Rolling AST,home_Rolling ST,home_Rolling TO,home_Rolling BL,home_Rolling PTS,home_Rolling AST/TO,home_Rolling 3P%,home_Rolling FG%,home_Rolling FT%,home_SPREAD
0,21300275,4/8/14,San Antonio Spurs,0,1,41.033333,8.966667,17.366667,35.066667,25.966667,...,24.500000,8.600000,13.600000,3.766667,108.033333,1.916725,0.333378,0.458320,0.766028,7.0
1,21300447,12/28/13,Utah Jazz,0,1,35.633333,6.366667,15.600000,29.066667,19.800000,...,23.566667,8.300000,13.500000,4.633333,105.300000,1.909397,0.333049,0.465288,0.719054,-14.5
2,21300455,12/30/13,Dallas Mavericks,0,1,39.466667,8.533333,16.466667,30.700000,23.233333,...,23.433333,9.300000,13.533333,2.866667,106.433333,1.954478,0.337522,0.429603,0.794908,-4.0
3,21300458,12/30/13,Charlotte Hornets,0,1,34.866667,5.033333,18.500000,33.233333,19.766667,...,20.200000,7.300000,14.366667,4.966667,93.133333,1.579379,0.346225,0.434064,0.771320,-2.5
4,21300460,12/31/13,Atlanta Hawks,0,1,39.000000,9.566667,15.866667,32.700000,25.800000,...,18.666667,6.800000,15.000000,4.566667,95.233333,1.347540,0.345794,0.454824,0.770221,-3.0
5,21300462,12/31/13,Golden State Warriors,0,1,38.533333,9.233333,15.700000,35.333333,22.566667,...,20.633333,7.800000,14.933333,4.433333,97.433333,1.449365,0.356744,0.450766,0.765446,6.0
6,21300465,12/31/13,Brooklyn Nets,0,1,34.800000,7.466667,19.900000,30.700000,20.300000,...,24.733333,8.100000,14.166667,4.466667,103.933333,1.949956,0.386288,0.484496,0.760387,-12.0
7,21300466,12/31/13,Portland Trail Blazers,0,1,40.166667,10.400000,18.433333,33.166667,24.166667,...,22.233333,7.966667,15.400000,6.300000,106.066667,1.517957,0.355699,0.472271,0.811866,-6.0
8,21300467,12/31/13,Milwaukee Bucks,0,1,34.800000,7.066667,16.000000,30.200000,20.733333,...,21.933333,6.800000,15.100000,5.933333,99.033333,1.540992,0.363242,0.438721,0.747852,-6.0
9,21300471,1/1/14,Philadelphia 76ers,0,1,39.500000,7.500000,16.166667,33.200000,23.066667,...,21.166667,7.233333,14.266667,5.766667,99.833333,1.637551,0.349364,0.441822,0.733192,-9.0


In [29]:
#df

#only retain the home flag, since we just care about the being home outcome since it aligns with home team spread
df['Home'] = df['home_Home']
df['GAME_ID'] = df['road_GAME_ID']
del df['road_GAME_ID'],df['home_GAME_ID'],df['road_Date'],df['home_Date']
del df['road_Away'],df['road_Home'],df['home_Away'],df['home_Home']

In [30]:
data = pd.read_csv('NBADATA.csv')
data = data[['GAME_ID','PLUS_MINUS','Home']]

In [40]:
df = df.merge(data,on=['GAME_ID','Home'])  #this is correct. 

KeyError: 'GAME_ID'

In [41]:
df.columns

#remove the final extraneous columns. 

del df['road_Team'],df['home_Team'],df['GAME_ID']
del df['Home']
#df['Home Team Covers?'] = 

KeyError: 'road_Team'

Quick brainstorm. This plus minus refers to the final point differential for the home team. If the point differential is smaller than the spread attributed to the home team, 

Ex. +- = -10. Means home team lost by 10. If spread was +11, then the team covere. Positive number

if +- = 24. If home spread was -26, they didn't cover spread negative number


so if +- plus spread >0, they covered. Else they didn/t 

In [42]:
df.columns

Index(['road_Rolling FG', 'road_Rolling 3P', 'road_Rolling FT',
       'road_Rolling DR', 'road_Rolling AST', 'road_Rolling ST',
       'road_Rolling TO', 'road_Rolling BL', 'road_Rolling PTS',
       'road_Rolling AST/TO', 'road_Rolling 3P%', 'road_Rolling FG%',
       'road_Rolling FT%', 'road_SPREAD', 'home_Rolling FG', 'home_Rolling 3P',
       'home_Rolling FT', 'home_Rolling DR', 'home_Rolling AST',
       'home_Rolling ST', 'home_Rolling TO', 'home_Rolling BL',
       'home_Rolling PTS', 'home_Rolling AST/TO', 'home_Rolling 3P%',
       'home_Rolling FG%', 'home_Rolling FT%', 'home_SPREAD'],
      dtype='object')

In [43]:
outcome = df['PLUS_MINUS'] #df['home_SPREAD'] + df['PLUS_MINUS'] 

KeyError: 'PLUS_MINUS'

In [44]:
y = []
for val in outcome:
    if val>0: 
        y.append(1)
    else:
        y.append(0)

In [45]:
del df['PLUS_MINUS']

KeyError: 'PLUS_MINUS'

In [46]:
X = df

In [47]:
len(X)

3850

In [48]:
X.head()

,road_Rolling FG,road_Rolling 3P,road_Rolling FT,road_Rolling DR,road_Rolling AST,road_Rolling ST,road_Rolling TO,road_Rolling BL,road_Rolling PTS,road_Rolling AST/TO,...,home_Rolling AST,home_Rolling ST,home_Rolling TO,home_Rolling BL,home_Rolling PTS,home_Rolling AST/TO,home_Rolling 3P%,home_Rolling FG%,home_Rolling FT%,home_SPREAD
0,41.033333,8.966667,17.366667,35.066667,25.966667,7.366667,14.200000,5.766667,108.400000,1.940596,...,24.500000,8.6,13.600000,3.766667,108.033333,1.916725,0.333378,0.458320,0.766028,7.0
1,35.633333,6.366667,15.600000,29.066667,19.800000,7.300000,14.233333,5.033333,93.233333,1.563940,...,23.566667,8.3,13.500000,4.633333,105.300000,1.909397,0.333049,0.465288,0.719054,-14.5
2,39.466667,8.533333,16.466667,30.700000,23.233333,9.533333,14.133333,4.700000,103.933333,1.840691,...,23.433333,9.3,13.533333,2.866667,106.433333,1.954478,0.337522,0.429603,0.794908,-4.0
3,34.866667,5.033333,18.500000,33.233333,19.766667,6.333333,12.700000,5.366667,93.266667,1.742615,...,20.200000,7.3,14.366667,4.966667,93.133333,1.579379,0.346225,0.434064,0.771320,-2.5
4,39.000000,9.566667,15.866667,32.700000,25.800000,8.300000,14.600000,4.466667,103.433333,1.921392,...,18.666667,6.8,15.000000,4.566667,95.233333,1.347540,0.345794,0.454824,0.770221,-3.0


In [49]:
y

[1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [50]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state = 0)

Now prior to normalizing, create a separate var that holds onto all of the spreads for the home teams in the testing games. 

In [51]:
test_spreads = X_test['home_SPREAD']

In [52]:
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [53]:
from sklearn.neural_network import MLPClassifier

from sklearn.svm import SVC
model = MLPClassifier()

model.fit(X_train,y_train)



MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [54]:
model.score(X_train,y_train)

0.69587807412538971

In [55]:
model.score(X_test,y_test)

0.67705088265835933

Now to compare: 

In [56]:
predictions = model.predict(X_test)


for i,prediction in enumerate(predictions):
   # print("Prediction: ", prediction)
    #print("Spread of the home team for this game: ", test_spreads.values[i])
    
    if test_spreads.values[i] > 0 and prediction ==1:
        print("Picked a home team underdog")
        
    if test_spreads.values[i] < 0 and prediction ==0:
        print("Picked a away team underdog")

Picked a home team underdog
Picked a home team underdog
Picked a home team underdog
Picked a home team underdog
Picked a home team underdog
Picked a home team underdog
Picked a home team underdog
Picked a home team underdog
Picked a home team underdog
Picked a home team underdog
Picked a home team underdog
Picked a home team underdog
Picked a home team underdog
Picked a home team underdog
Picked a home team underdog
Picked a home team underdog
Picked a home team underdog
Picked a home team underdog
Picked a home team underdog
Picked a home team underdog
Picked a home team underdog
Picked a home team underdog
Picked a home team underdog
Picked a home team underdog
Picked a home team underdog
Picked a home team underdog
Picked a home team underdog
Picked a home team underdog
Picked a home team underdog
Picked a home team underdog
Picked a home team underdog
Picked a home team underdog
Picked a home team underdog
Picked a away team underdog
Picked a home team underdog
Picked a home team u

This is a good time to note this seems to very rarely pick underdogs, but more often it does on the road.  -- OK now the model picks the opposite, let's come back to this later after a proper payout calculator is made at least. 

In [58]:
def spread2ML(spread):
    """Converts spread into a moneyline value using the equation I derived in another notebook. 
    """
    
    ML = 0.5073158 * spread**3 + -1.09055921 * spread**2 + 24.65777543 * spread - 16.06823207
    
    return ML


def ML2Payout(ML,bet,win=True):
    """Convert Moneyline odds to a payout. 
    """
    if win:
        if ML < 0: # - moneyline, 
        # PAYOUT = BET AMOUNT / (-1 *MONEYLINE ODDS / 100)

            payout = bet / (-1*ML/100)

        elif ML > 0:   #now for the underdog
        #PAYOUT = BET AMOUNT * ODDS / 100
            payout = (bet * ML) / 100

            
        else:
            payout = bet
    else:
        if ML > 0: 
            payout = -bet
        elif ML < 0:
            #in the circumstances where its a favorite, the computer makes you put down more. ie -190 means 19 to win 10. 
            payout = -bet
            
        else:
            payout = -bet
    
    return payout 

In [59]:
ML2Payout(-190,10,win=True)

5.2631578947368425

In [67]:
def risk2payout(ML,bet,win=True):
    """Depending on the moneyline, the risk reward formula changes. 
    """
    
    if ML < 0: # if betting on a favorite. 
        
        risk = -ML/bet
        reward = bet
        if win:
            payout = reward
        else:
            payout = -risk
        
    if ML > 0: #if betting on an underdog. 
        risk = bet
        reward = ML/bet
        
        if win:
            payout = reward #this is your risked money back, plus the reward. 
        else:
            payout = -risk  #this is how much you risked, and it's gone. 
    
    return payout
        
        
        
        
        

In [68]:
risk2payout(660,10,win=True)

66.0

In [ ]:
money_made = 0

for i,prediction in enumerate(predictions):
   # print("Prediction: ", prediction)
    #print("Spread of the home team for this game: ", test_spreads.values[i])
    
    #prediction
    
    
    spread = test_spreads.values[i]
    ML_odds = spread2ML(spread)
    
    init_bet = 1
    
   # ML * 
    print(spread)

    if y_test[i] == prediction:
        print("you won! ")
        
        winnings = risk2payout(ML_odds,init_bet,win=True)
        
       # print('$',winnings)
        
        money_made += winnings
       # if ML_odds < -
    else:
       # print("you lost :/ ")
        print('$ ', losings)
        losings = risk2payout(ML_odds,init_bet,win=False)

        money_made += losings
     #   print("xxx")
    
    
  #  if test_spreads.values[i] > 0 and prediction ==1:
   #     print("Picked a home team underdog")
  #      
  #  if test_spreads.values[i] < 0 and prediction ==0:
   #     print("Picked a away team underdog")

In [ ]:
money_made

In [ ]:
spread2ML(max(test_spreads))

# So considering the spread along with these rolling statistics, able to successfully pick the winner of NBA games with approximately 69% accuracy. By also considering moneyline odds converted from spread, how much money do you win? 

# Quick Review

# 1. Identified stats most correlated to point differential in a game.

# 2. Identified ngamesplits most correlated to performance in said game. 

# 3. Trained a model on these statistics and found can pick winners with ~68% accuracy, and spread still barely over 50%. 

# What's up for next time? Refactor this code, and turn into an updateable model as more games come in using those scrapers, and also provide a fast application for guess and check. Also convert between traditional spread to create a converter for money made. 